# Cerrado Physiognomy Classification - Random Forest Classifier

#### Data treatment

In [1]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv("data.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468 entries, 0 to 467
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   references  468 non-null    object 
 1   vegetation  468 non-null    object 
 2   local       468 non-null    object 
 3   family      465 non-null    object 
 4   specie      468 non-null    object 
 5   month       468 non-null    object 
 6   nitrogen    463 non-null    float64
 7   phosphorus  468 non-null    float64
 8   potassium   468 non-null    float64
 9   calcium     468 non-null    float64
 10  magnesium   468 non-null    float64
 11  iron        398 non-null    float64
 12  manganese   398 non-null    float64
 13  zinc        233 non-null    float64
 14  copper      233 non-null    float64
 15  aluminum    468 non-null    int64  
dtypes: float64(9), int64(1), object(6)
memory usage: 58.6+ KB


In [6]:
df.head()

,references,vegetation,local,family,specie,month,nitrogen,phosphorus,potassium,calcium,magnesium,iron,manganese,zinc,copper,aluminum
0,"Ribeiro, J.F. (1983)",Cerrado,"CPAC, EMBRAPA",1.21,Qualea parviflora Mart.,may,12.1,0.56,3.7,5.3,1.9,112.0,194.0,15.9,4.16,14353
1,"Ribeiro, J.F. (1983)",Cerrado,"CPAC, EMBRAPA",1.31,Qualea grandiflora Mart.,may,13.1,0.67,4.2,5.0,2.0,118.0,118.0,16.6,3.13,5500
2,"Ribeiro, J.F. (1983)",Cerrado,"CPAC, EMBRAPA",1.56,Annona crassiflora Mart.,may,15.6,0.76,5.4,6.4,1.7,77.0,82.0,18.5,3.33,184
3,"Ribeiro, J.F. (1983)",Cerrado,"CPAC, EMBRAPA",1.39,Caryocar brasiliense A.St.-Hil.,may,13.9,0.72,2.9,5.2,2.3,153.0,60.0,15.6,2.13,310
4,"Ribeiro, J.F. (1983)",Cerrado,"CPAC, EMBRAPA",2.62,Dimorphandra mollis Benth.,may,26.2,0.40,3.0,1.5,1.0,184.0,18.0,15.2,1.97,410


In [16]:
df2 = df.copy()

In [ ]:
for d in df2:
    print(df2[d].value_counts())

In [61]:
# Selecting the features
df3 = df2[["vegetation", "nitrogen", "phosphorus", "potassium", "calcium"]]

In [57]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468 entries, 0 to 467
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   vegetation  468 non-null    object 
 1   nitrogen    463 non-null    float64
 2   phosphorus  468 non-null    float64
 3   potassium   468 non-null    float64
 4   calcium     468 non-null    float64
dtypes: float64(4), object(1)
memory usage: 18.4+ KB


In [62]:
# Dropping rows with NAN
df_index = df3.index[df3.isnull().any(axis=1)]
df3.drop(df_index, 0, inplace=True)

/tmp/ipykernel_7934/104729926.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df3.drop(df_index, 0, inplace=True)
/tmp/ipykernel_7934/104729926.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.drop(df_index, 0, inplace=True)


In [63]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463 entries, 0 to 467
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   vegetation  463 non-null    object 
 1   nitrogen    463 non-null    float64
 2   phosphorus  463 non-null    float64
 3   potassium   463 non-null    float64
 4   calcium     463 non-null    float64
dtypes: float64(4), object(1)
memory usage: 21.7+ KB


In [64]:
df3["vegetation"].unique()

array(['Cerrado', 'Cerradão', 'Mata', 'Mata Mes. Semidec.',
       'campo cerrado', 'campo sujo', 'mata semidecícua',
       'cerradão mesotrófico', 'cerrado sensu stricto', 'cerrado denso',
       'cerradão distrófico'], dtype=object)

In [30]:
names = {
    "Cerrado": "cerrado",
    "Cerradão": "distrofic_cerradao",
    "Mata": "mesotrofic_forest",
    "Mata Mes. Semidec.": "mesotrofic_forest",
    "campo cerrado": "campo_cerrado",
    "campo sujo": "campo_sujo",
    "mata semidecícua": "mesotrofic_forest",
    "cerradão mesotrófico": "mesotrofic_cerradao",
    "cerrado sensu stricto": "cerrado",
    "cerrado denso": "distrofic_cerradao",
    "cerradão distrófico": "distrofic_cerradao"
}

In [65]:
# Replacing value names on vegetation column
df3.replace({"vegetation": names}, inplace=True)

/tmp/ipykernel_7934/2612624977.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.replace({"vegetation": names}, inplace=True)


In [66]:
df3["vegetation"].unique()

array(['cerrado', 'distrofic_cerradao', 'mesotrofic_forest',
       'campo_cerrado', 'campo_sujo', 'mesotrofic_cerradao'], dtype=object)

In [67]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463 entries, 0 to 467
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   vegetation  463 non-null    object 
 1   nitrogen    463 non-null    float64
 2   phosphorus  463 non-null    float64
 3   potassium   463 non-null    float64
 4   calcium     463 non-null    float64
dtypes: float64(4), object(1)
memory usage: 21.7+ KB


#### Creating the model